# SLAKH Dataset Working

In [5]:
import os
import yaml
from benedict import benedict
import matplotlib.pyplot as plt
import torch
import mirdata
import time

%load_ext autoreload
%autoreload 2

from dataloader_slakh import Target, Track, SlakhDataset

In [3]:
slakh_dataset = SlakhDataset()

# 0.036s for loading of stem audio
# 0.097s for loading of mix audio

From cffi callback <function SoundFile._init_virtual_io.<locals>.vio_tell at 0x7f1324279050>:
Traceback (most recent call last):
  File "/home/olaf/Documents/GitHub/ddspzart/ws_2/.openunmix_venv/lib/python3.7/site-packages/soundfile.py", line 1232, in vio_tell
    @_ffi.callback("sf_vio_tell")
KeyboardInterrupt


In [14]:
slakh_dataset.target = "Trumpet"
slakh_dataset.filter_target()
print(slakh_dataset.tracks[0].targets[0].instrument)
# print(slakh_dataset.targets)

Fretless Bass


In [47]:
for i in range(5):
    slakh_dataset[0][0].shape

In [ ]:
for i in range(len(slakh_dataset.tracks)):
    print(slakh_dataset.tracks[i]," -> ",[slakh_dataset.tracks[i].targets[j].instrument for j in range(len(slakh_dataset.tracks[i].targets))])

## Functions helpful for filtering the dataset

In [2]:
DATASET_FOLDER = "/media/olaf/OlafSSD/03_Dokumente/01_epfl/01_cm/slakh2100_flac_redux/train"

track_folders = sorted(os.listdir(DATASET_FOLDER))
stem_audio_paths = {}
mix_audio_paths = {}

instruments = []
instruments_folder = []

for track_folder in track_folders:
    d = benedict.from_yaml(DATASET_FOLDER + "/"+ track_folder + "/metadata.yaml")
    instruments_stem = []
    mix_audio_paths[track_folder[-5:]] = DATASET_FOLDER + "/" + track_folder + "/mix.wav"
    stems = {}
    for stem in sorted(d["stems"].keys()):
        stems[stem] = DATASET_FOLDER + "/" + track_folder + "/stems/" + stem + ".wav"
        instrument = d["stems"][stem]["midi_program_name"]
        instruments.append(instrument)
        instruments_stem.append(instrument)

    # print(track_folder,instruments_stem)
    instruments_folder.append(instruments_stem)
    stem_audio_paths[track_folder[-5:]] = stems

In [3]:
# print(sorted(set(instruments)))
interesting_instrs = ['Flute','French Horn', 'Bassoon','Clarinet','Electric Guitar (jazz)','Viola', 'Violin', 'Trombone', 'Trumpet', 'Tuba','Alto Sax','Tenor Sax']
indxs = [[] for i in range(len(interesting_instrs))]
interesting_instrs_indxs = dict(zip(interesting_instrs, indxs))
for inst in interesting_instrs:
    interesting_instrs_indxs[inst] = [i for i, x in enumerate(instruments) if x == inst]
    print(inst,len(interesting_instrs_indxs[inst]))
instruments_folder

Flute 172
French Horn 79
Bassoon 9
Clarinet 54
Electric Guitar (jazz) 392
Viola 11
Violin 60
Trombone 85
Trumpet 81
Tuba 21
Alto Sax 204
Tenor Sax 97


[['Distortion Guitar',
  'Drums',
  'Bright Acoustic Piano',
  'Electric Bass (finger)',
  'Choir Aahs',
  'Percussive Organ',
  'FX 8 (sci-fi)',
  'Electric Guitar (jazz)',
  'Distortion Guitar',
  'Harmonica',
  'Harmonica'],
 ['Synth Strings 1',
  'Electric Bass (pick)',
  'Lead 2 (sawtooth)',
  'Pad 1 (new age)',
  'Choir Aahs',
  'Music Box',
  'Clavinet',
  'Brass Section',
  'Electric Guitar (muted)',
  'Drums',
  'Woodblock',
  'Orchestra Hit'],
 ['Bright Acoustic Piano',
  'Electric Bass (pick)',
  'Acoustic Guitar (steel)',
  'Overdriven Guitar',
  'Electric Guitar (jazz)',
  'Fiddle',
  'Drums',
  'Electric Guitar (clean)',
  'Acoustic Guitar (steel)'],
 ['Pad 3 (polysynth)',
  'Electric Bass (finger)',
  'Choir Aahs',
  'Rock Organ',
  'Electric Guitar (muted)',
  'Synth Strings 1',
  'Harpsichord',
  'Acoustic Grand Piano',
  'Drums'],
 ['Drums',
  'Acoustic Guitar (steel)',
  'Acoustic Grand Piano',
  'Pan Flute',
  'Electric Bass (finger)',
  'Glockenspiel',
  'Electric 

In [ ]:
print(set(instruments))
plt.hist(instruments)

In [ ]:
dataset = mirdata.initialize('slakh',"../datasets")
slakh_data = dataset.load_tracks()
print(mirdata.list_datasets())
print(dataset.remotes)
# dataset.download(['baby'])
print(dataset.get_mtrack_splits()['test'][:3])
print(dataset.get_track_splits()['train'][:3])
slakh_data['Track00001-S01'].instrument

In [ ]:
slakh_data = dataset.load_tracks()
tic = time.perf_counter()
for i in range(1000):
    x,sr = mirdata.datasets.slakh.load_audio(stem_audio_paths["00001"]["S00"])
toc = time.perf_counter()
print(f"{toc-tic:.4}")
plt.plot(x)

In [21]:
# Functionality to create subset of SLAKH dataset with all Tracks for one instrument to be able to load it on the cluster.

import shutil

DATASET_FOLDER = "/media/olaf/OlafSSD/03_Dokumente/01_epfl/01_cm/slakh2100_flac_redux"
DATASET_FOLDER_FLUTE = "/media/olaf/OlafSSD/03_Dokumente/01_epfl/01_cm/slakh2100_flac_redux_flute"

split_folders = sorted(os.listdir(DATASET_FOLDER))
print(split_folders)
# split_folders = [s for s in split_folders if s is not "README" or s is not "omitted"]
for split in split_folders[2:]:
    print(split)
    os.makedirs(DATASET_FOLDER_FLUTE+"/"+split) if not os.path.exists(DATASET_FOLDER_FLUTE+"/"+split) else print("exists")
    track_folders = sorted(os.listdir(DATASET_FOLDER+"/"+split))

    for track_folder in track_folders[:]:
        print(track_folder)
        d = benedict.from_yaml(DATASET_FOLDER +"/"+split + "/"+ track_folder + "/metadata.yaml")
        start = True
        for stem in sorted(d["stems"].keys()):
            instrument = d["stems"][stem]["midi_program_name"]
            if instrument == "Flute":
                if start:
                    os.makedirs(DATASET_FOLDER_FLUTE+"/"+split+"/"+track_folder) if not os.path.exists(DATASET_FOLDER_FLUTE+"/"+split+"/"+track_folder) else print("exists")
                    os.makedirs(DATASET_FOLDER_FLUTE+"/"+split+"/"+track_folder+"/stems") if not os.path.exists(DATASET_FOLDER_FLUTE+"/"+split+"/stems/"+track_folder) else print("exists")
                    src = DATASET_FOLDER+"/"+split+"/"+track_folder+"/mix.flac"
                    dst = DATASET_FOLDER_FLUTE+"/"+split+"/"+track_folder+"/mix.flac"
                    shutil.copyfile(src, dst)
                    src = DATASET_FOLDER+"/"+split+"/"+track_folder+"/metadata.yaml"
                    dst = DATASET_FOLDER_FLUTE+"/"+split+"/"+track_folder+"/metadata.yaml"
                    shutil.copyfile(src, dst)
                    start = False
                try:
                    src = DATASET_FOLDER+"/"+split+"/"+track_folder + "/stems/" + stem + ".flac"
                    dst = DATASET_FOLDER_FLUTE+"/"+split+"/"+track_folder + "/stems/" + stem + ".flac"
                    shutil.copyfile(src, dst)
                except:
                    print("Did not work", split,track_folder,stem)

['README', 'omitted', 'test', 'train', 'validation']
test
Track01876
Track01877
Track01878
Track01880
Track01881
Track01882
Track01883
Track01884
Track01886
Track01887
Track01888
Track01889
Track01891
Track01892
Track01893
Track01895
Track01896
Track01897
Track01898
Track01899
Track01900
Track01901
Track01902
Track01903
Track01904
Track01905
Track01906
Track01907
Track01908
Track01911
Track01913
Track01916
Track01917
Track01918
Track01920
Track01925
Track01927
Track01928
Track01929
Track01930
Track01931
Track01932
Track01934
Track01935
Track01936
Track01937
Track01940
Track01943
Track01945
Track01947
Track01948
Track01949
Track01950
Track01951
Track01952
Track01954
Track01955
Track01956
Track01957
Track01959
Track01960
Track01961
Track01962
Track01963
Track01965
Track01967
Track01968
Track01972
Track01973
Track01974
Track01975
Track01976
Track01977
Track01978
Track01980
Track01981
Track01982
Track01985
Track01986
Track01987
Track01989
Track01990
Track01993
Track01994
Track01995
Track01

FileNotFoundError: [Errno 2] No such file or directory: '/media/olaf/OlafSSD/03_Dokumente/01_epfl/01_cm/slakh2100_flac_redux/test/Track02001/stems/S06.flac'